# Assign NPIs!

In [3]:
%load_ext google.cloud.bigquery
import pandas as pd
from sodapy import Socrata
from npyi import npi

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


Get the 2016 open payments dataset

In [5]:
%%bigquery
SELECT
    source_year AS year,
    COUNT(is_male) AS birth_count
FROM `bigquery-public-data.samples.natality`
GROUP BY year
ORDER BY year DESC
LIMIT 15

,year,birth_count
0,2008,4255156
1,2007,4324008
2,2006,4273225
3,2005,4145619
4,2004,4118907
5,2003,4096092
6,2002,4027376
7,2001,4031531
8,2000,4063823
9,1999,3963465


In [14]:
%%bigquery df
SELECT 
    Covered_Recipient_Type, 
    Teaching_Hospital_Name, 
    Physician_First_name, 
    Physician_Last_Name,
    Recipient_Primary_Business_Street_Address_Line1, 
    Recipient_City, 
    Recipient_State, 
    Recipient_State
FROM `carbide-server-294318.op2016.OP2016`

In [ ]:
# Insert NPI column
df['NPI'] = 

In [17]:
# Organization example
response = npi.search(search_params={'organization_name': 'PRESBYTERIAN HOSPITAL',
                                     'city': 'ALBUQUERQUE'})
#print(response.keys()) # dict_keys(['result_count', 'results'])
response

{'result_count': 1,
 'results': [{'enumeration_type': 'NPI-2',
   'number': 1215913470,
   'last_updated_epoch': 1557420354,
   'created_epoch': 1135036800,
   'basic': {'organization_name': 'PRESBYTERIAN HEALTHCARE SERVICES',
    'organizational_subpart': 'NO',
    'enumeration_date': '2005-12-20',
    'last_updated': '2019-05-09',
    'status': 'A',
    'authorized_official_first_name': 'KIMBERLY',
    'authorized_official_last_name': 'POLAND',
    'authorized_official_telephone_number': '505-923-5355',
    'authorized_official_title_or_position': 'MANAGER',
    'name': 'PRESBYTERIAN HEALTHCARE SERVICES'},
   'other_names': [{'organization_name': 'PRESBYTERIAN HOSPITAL',
     'code': '3',
     'type': 'Doing Business As'}],
   'addresses': [{'country_code': 'US',
     'country_name': 'United States',
     'address_purpose': 'LOCATION',
     'address_type': 'DOM',
     'address_1': '1100 CENTRAL SE',
     'address_2': '',
     'city': 'ALBUQUERQUE',
     'state': 'NM',
     'postal_co

In [15]:
# Individual example
response = npi.search(search_params={'first_name': 'Andrew', 'last_name': 'Jackson'})
#print(response.keys()) # dict_keys(['result_count', 'results'])
response

{'result_count': 10,
 'results': [{'enumeration_type': 'NPI-1',
   'number': 1578707204,
   'last_updated_epoch': 1240531200,
   'created_epoch': 1240531200,
   'basic': {'first_name': 'ANDREA',
    'last_name': 'JACKSON',
    'middle_name': 'DENISE',
    'credential': 'DDS, MS',
    'sole_proprietor': 'YES',
    'gender': 'F',
    'enumeration_date': '2009-04-24',
    'last_updated': '2009-04-24',
    'status': 'A',
    'name': 'JACKSON ANDREA'},
   'other_names': [{'code': '2',
     'type': 'Professional Name',
     'last_name': 'JACKSON',
     'first_name': 'ANDREA',
     'middle_name': 'D',
     'prefix': 'DR.',
     'credential': 'DDS, MS'}],
   'addresses': [{'country_code': 'US',
     'country_name': 'United States',
     'address_purpose': 'LOCATION',
     'address_type': 'DOM',
     'address_1': '6912 WADE AVE',
     'address_2': '',
     'city': 'CLINTON',
     'state': 'MD',
     'postal_code': '207352743',
     'telephone_number': '301-868-3317'},
    {'country_code': 'US',

In [ ]:
# NPI matching

# for loop to go through every row in the OP dataframe
# Source 3
for index, row in df.iterrows():
    
    # Is the provider individual or enterprise?
    if row.Covered_Recipient_Type == 'Covered Recipient Teaching Hospital'
        # Source 2
        results = npi.search(search_params={'organization_name': row.Teaching_Hospital_Name,
                                            'address_purpose': 'PRIMARY',
                                            'city': row.recipient_city,
                                            'state': row.recipient_state})                       
        
    if row.Covered_Recipient_Type == 'Covered Recipient Physician'
        # Source 2
        results = npi.search(search_params={'first_name': row.physician_first_name,
                                            'last_name': row.physcian_last_name,
                                            'address_purpose': 'PRIMARY',
                                            'city': row.recipient_city,
                                            'state': row.recipient_state})

    # Iterate through the list of matches to find the exact match
    matches = 0
    for j in range(results['result_count']):
        if results['results'][i]['addresses'][0]['address_1'] == row.recipient_primary_business_street_address_1
            # We have a match!
            matches += 1
    
    # Check to see if we only found one match
    if matches == 1:
        row.NPI = results['results'][i]['number']

In [ ]:
# Organization search
if df.Covered_Recipient_Type == 'Covered Recipient Teaching Hospital'
    # Source 2
    npi.search(search_params={'organization_name': df.Teaching_Hospital_Name,
                              'address_purpose': 'PRIMARY',
                              'city': df.recipient_city,
                              'state': df.recipient_state})

In [ ]:
# Individual search
if df.Covered_Recipient_Type == 'Covered Recipient Physician'
    # Source 2
    npi.search(search_params={'first_name': df.physician_first_name, 
                              'last_name': df.physcian_last_name,
                              'address_purpose': 'PRIMARY',
                              'city': df.recipient_city,
                              'state': df.recipient_state})

In [ ]:
# Search resulting list for exact match using the addresses
for i in ?['result_count']:
    if ?['results'][i]['addresses'][0]['address_1'] == df.recipient_primary_business_street_address_1
        # We have a match!
        df.NPI = ?['results'][i]['number']

In [16]:
npi.VALID_SEARCH_PARAMS

('number',
 'enumeration_type',
 'taxonomy_description',
 'first_name',
 'use_first_name_alias',
 'last_name',
 'organization_name',
 'address_purpose',
 'city',
 'state',
 'postal_code',
 'country_code')

Sources
1. https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/56014
2. https://npyi.readthedocs.io/en/latest/
3. https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas